In [ ]:
import pyvelox.pyvelox as pv
import time
import statistics
import duckdb
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
dataLocation = "../../adult/discrete.csv"
queryLocation = "../../adult/queries_100.txt"

millisecondMultiplier = 1000

In [ ]:
queries = []
with open(queryLocation, "r") as file_in:
        for line in file_in:
            line = line.strip()
            queries.append(line)

print(queries)

In [ ]:
con = duckdb.connect(database=':memory:')
con.execute('CREATE TABLE data AS SELECT * FROM read_csv_auto(\'' + dataLocation +'\')')
con.execute('SET threads TO 1')

resultSizes = []

for query in queries:
    result = con.execute(query).fetchall()
    resultSizes.append(len(result))

print(resultSizes)

plotData = np.array(resultSizes)

plt.hist(plotData, 10)

plt.xlabel('Result Size')
plt.ylabel('')
plt.grid(axis='y', alpha=0.75)

plt.show()

DuckDB execution

In [ ]:
con = duckdb.connect(database=':memory:')
con.execute('CREATE TABLE data AS SELECT * FROM read_csv_auto(\'' + dataLocation +'\')')
con.execute('SET threads TO 1')

duckTimes = []

for i in range(0, 5):
    resultsDuck = []

    start_time = time.time()
    for query in queries:
        resultsDuck.append(con.execute(query).fetchall())
    end_time = time.time()
    res = end_time - start_time
    duckTimes.append(res * millisecondMultiplier)


print('mean execution: ', statistics.mean(duckTimes))
print('std execution: ', statistics.stdev(duckTimes))

Project execution

In [ ]:
proj = pv.Project.create()
proj.loadData(dataLocation)
proj.setNumThreads(1)

projectTimes = []

for i in range(0, 5):
    resultsProj = []

    st = time.time()
    exprs = proj.compileSQLQueries(queries)  
    for expr in exprs:
        resultsProj.append(proj.runQueryOneThread(expr, 0))
    et = time.time()

    res = et - st
    final_res = res * millisecondMultiplier
    projectTimes.append(final_res)


print('Execution time mean:', statistics.mean(projectTimes), 'milliseconds')
print('Execution time std:', statistics.stdev(projectTimes), 'milliseconds')

Checking corectness

In [ ]:
areResultsEqual = True
notPrinted = True
for i in range(0, len(queries)):
    areResultsEqual = areResultsEqual and (len(resultsProj[i]) == len(resultsDuck[i]))
    for j in range(0, len(resultsDuck[i])):
        areResultsEqual = areResultsEqual and (resultsProj[i][j] == resultsDuck[i][j][0])
        if not areResultsEqual and notPrinted:
            notPrinted = False
            print(i)
            print(j)

print("Results are equal: " + str(areResultsEqual))

Applying mutations

In [ ]:
nbEpochs = 5
comparisonOperators = ["eq", "neq", "gte", "lte", "gt", "lt"]
conjucntionOperators = ["and", "or"]

def applyMutations(expr):
    inputs = []
    label = ""
    for input in expr.getInputs():
        inputs.append(applyMutations(input))

    if bool(random.getrandbits(1)) and expr.isCallExpr():
        call = expr.toCallExpr()
        name = call.getName()
        if name in conjucntionOperators:
            return call.mutate(random.choice(conjucntionOperators), inputs)
        if name in comparisonOperators:
            return call.mutate(random.choice(comparisonOperators), inputs)
        
    return expr.copyWInputs(inputs)

In [ ]:

for i in range(0, 5):
    mutants = []

    for epoch in range(0, nbEpochs):
        queries = []
        for q in range(0, len(exprs)):
            st = time.time()
            mutant = applyMutations(exprs[q])
            proj.runQueryOneThread(mutant, 0)
            et = time.time()
            mutants.append(mutant.getSQL())
            projectTimes[i] += (et - st) * millisecondMultiplier


    st = time.time()
    for mutant in mutants:
        con.execute("SELECT id FROM data WHERE " + mutant).fetchall()
    et = time.time()
    duckTimes[i] += (et - st) * millisecondMultiplier

print('Project Execution time mean:', statistics.mean(projectTimes), 'milliseconds')
print('Project Execution time std:', statistics.stdev(projectTimes), 'milliseconds')
print('DuckDB Execution time mean:', statistics.mean(duckTimes), 'milliseconds')
print('DuckDB Execution time std:', statistics.stdev(duckTimes), 'milliseconds')
print('Improvement: ', (1 - statistics.mean(projectTimes) / statistics.mean(duckTimes)) * 100, '%')